In [1]:
from phantom.common.distribute import HighsPyProblem

In [2]:
size = 256
keys = [(i, i) for i in range(size)]

In [3]:
lookup_dict = {k: k for k in keys}

In [4]:
from functools import cache


@cache
def lookup_cache(key):
    return lookup_dict.get(key)

In [5]:
import numpy as np

lookup_array = np.zeros((size, size), dtype="object")
for k in keys:
    lookup_array[k] = k

In [6]:
%%timeit
lookup_dict.get((100, 100))

28.7 ns ± 0.598 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [7]:
%%timeit
lookup_cache((100, 100))

46.1 ns ± 1.92 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [8]:
%%timeit
lookup_array[100, 100]

36.1 ns ± 1.38 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [10]:
%%timeit
HighsPyProblem(60, 60, True)

73.6 ms ± 981 μs per loop (mean ± std. dev. of 7 runs, 10 loops each)


In [11]:
problem = HighsPyProblem(60, 60, True)

In [12]:
%%timeit
problem.solve(np.ones((60, 60)), np.ones(60))

1.11 ms ± 11.6 μs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
